In [16]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [2]:
def read_compos(file='compo.json'):
    compos = json.load(open(file))['compos']
    df = pd.DataFrame(columns=['id', 'column_min', 'column_max', 'row_min', 'row_max', 'center', 'height', 'width', 'area', 'class'])
    for i, compo in enumerate(compos):
        compo['area'] = compo['height'] * compo['width']
        compo['center'] = ((compo['column_min'] + compo['column_max'])/2, (compo['row_min'] + compo['row_max'])/2)
        df.loc[i] = compo
    df = df[df['class'] != 'TextView']
    return df

In [3]:
def draw(org, compos, opt='class'):
    img_h, img_w = compos.iloc[0].height, compos.iloc[0].width
    img = cv2.resize(org, (img_w, img_h))
    board = img.copy()
    for i in range(len(compos)):
        compo = compos.iloc[i]
        board = cv2.rectangle(board, (compo.column_min, compo.row_min), (compo.column_max, compo.row_max), (255,0,0))
        board = cv2.putText(board, str(compo[opt]), (compo.column_min+5, compo.row_min+20), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,0,255), 1)
    
    cv2.imshow('board', board)
    cv2.waitKey()
    cv2.destroyAllWindows()

In [4]:
def inspect_data(compos):
    import matplotlib.pyplot as plt
    plt.xlim(xmax=10000)
    plt.scatter(list(compos['area']), list(compos['id']))

In [9]:
def dbscan_clustering_by_area(org, compos):
    x = np.reshape(list(compos['area']), (-1, 1))
    from sklearn.cluster import DBSCAN
    clustering = DBSCAN(eps=200, min_samples=1).fit(x)
    clustering.labels_
    compos['cluster'] = clustering.labels_
    draw(org, compos, 'cluster')
    return max(compos['cluster'])

In [10]:
compos = read_compos()
org = cv2.imread('9.png')
# draw(org, compos)

In [22]:
groups = compos.groupby('cluster').groups
for g in groups:
    print(groups[g])
    for i in groups[g]:
        print(compos.loc[i].center)

Int64Index([0], dtype='int64')
(317.5, 400.0)
Int64Index([1], dtype='int64')
(316.5, 5.0)
Int64Index([3], dtype='int64')
(283.5, 31.0)
Int64Index([4, 5, 6, 16, 26, 28, 29, 30, 31, 32, 33, 34, 83], dtype='int64')
(529.0, 31.5)
(582.0, 32.0)
(38.0, 58.0)
(468.5, 182.0)
(550.0, 226.5)
(509.0, 247.0)
(549.5, 243.0)
(470.0, 243.5)
(590.0, 243.5)
(469.0, 256.5)
(545.5, 256.5)
(587.5, 256.5)
(228.0, 774.0)
Int64Index([14], dtype='int64')
(535.5, 128.5)
Int64Index([15], dtype='int64')
(228.0, 153.5)
Int64Index([19, 20, 21, 22, 23], dtype='int64')
(62.5, 243.5)
(145.5, 243.5)
(228.0, 243.5)
(310.5, 243.5)
(393.5, 243.5)
Int64Index([40, 49, 57, 66, 71, 76, 80], dtype='int64')
(74.5, 319.0)
(74.5, 388.0)
(74.5, 456.0)
(74.5, 524.5)
(74.5, 593.0)
(74.5, 661.0)
(74.5, 730.0)
Int64Index([42, 47, 52], dtype='int64')
(476.5, 316.5)
(476.5, 356.5)
(476.5, 395.5)
Int64Index([84], dtype='int64')
(521.5, 137.5)
Int64Index([85], dtype='int64')
(350.5, 152.5)
